In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from statsmodels.othermod.betareg import BetaModel

# 1. 读取数据
df = pd.read_csv('data_processed_std.csv')

y_col = 'Y染色体浓度'
x_cols = ['检测孕周_周数','年龄','孕妇BMI','IVF妊娠','怀孕次数','生产次数']

# 3. 调整 y 到 (0,1)
eps = 1e-6
df['y_adj'] = df[y_col].clip(lower=eps, upper=1 - eps)

# 4. 划分数据
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# 5. 建模
formula = 'y_adj ~ ' + ' + '.join(x_cols)
model = BetaModel.from_formula(formula, data=train_df)
res = model.fit()
print(res.summary())

# 6. 预测与评估
y_pred = res.predict(test_df)
r2 = r2_score(test_df['y_adj'], y_pred)
mae = mean_absolute_error(test_df['y_adj'], y_pred)
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")


                              BetaModel Results                               
Dep. Variable:                  y_adj   Log-Likelihood:                 1279.2
Model:                      BetaModel   AIC:                            -2542.
Method:            Maximum Likelihood   BIC:                            -2507.
Date:                Fri, 05 Sep 2025                                         
Time:                        19:44:40                                         
No. Observations:                 613                                         
Df Residuals:                     605                                         
Df Model:                           6                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.4484      0.043    -56.449      0.000      -2.533      -2.363
检测孕周_周数        0.0324      0.019      1.729      0.0

In [ ]:
import pandas as pd
import numpy as np
import patsy
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error
from statsmodels.othermod.betareg import BetaModel

# ========== 1. 读取数据 ==========
df = pd.read_csv('data_processed_std.csv')



y_col = 'Y染色体浓度'
x_cols = ['检测孕周_周数','年龄','孕妇BMI','IVF妊娠','怀孕次数','生产次数',
          ]

# ========== 3. 调整 y 到 (0,1) ==========
eps = 1e-6
df['y_adj'] = df[y_col].clip(lower=eps, upper=1 - eps)

# ========== 4. 特征选择（不使用LassoCV） ==========
selected_features = x_cols

# ========== 5. 构造公式（均值方程用样条） ==========
# 对孕周使用 B-spline 样条（4 自由度）
mean_formula = 'y_adj ~ bs(检测孕周_周数, df=4) + ' + ' + '.join([f for f in selected_features if f != '检测孕周_周数'])

# 精度方程（假设精度随孕周和BMI变化）
precision_formula = '~ 检测孕周_周数 + 孕妇BMI'

# ========== 6. 交叉验证评估 ==========
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores, mae_scores = [], []

for train_idx, test_idx in kf.split(df):
    train_df, test_df = df.iloc[train_idx], df.iloc[test_idx]
    
    # 构造精度方程矩阵
    Z_train = patsy.dmatrix(precision_formula, train_df, return_type='dataframe')
    Z_test = patsy.dmatrix(precision_formula, test_df, return_type='dataframe')
    
    # 建模
    model = BetaModel.from_formula(mean_formula, data=train_df, exog_precision=Z_train)
    res = model.fit()
    
    # 预测
    y_pred = res.predict(test_df, exog_precision=Z_test)
    
    # 评估
    r2_scores.append(r2_score(test_df['y_adj'], y_pred))
    mae_scores.append(mean_absolute_error(test_df['y_adj'], y_pred))

print(f"平均 R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"平均 MAE: {np.mean(mae_scores):.4f} ± {np.std(mae_scores):.4f}")

# ========== 7. 最终模型拟合（全数据） ==========
Z_full = patsy.dmatrix(precision_formula, df, return_type='dataframe')
final_model = BetaModel.from_formula(mean_formula, data=df, exog_precision=Z_full)
final_res = final_model.fit()
print(final_res.summary())


平均 R²: 0.0353 ± 0.0648
平均 MAE: 0.0256 ± 0.0014
                              BetaModel Results                               
Dep. Variable:                  y_adj   Log-Likelihood:                 1845.4
Model:                      BetaModel   AIC:                            -3665.
Method:            Maximum Likelihood   BIC:                            -3603.
Date:                Fri, 05 Sep 2025                                         
Time:                        19:43:01                                         
No. Observations:                 877                                         
Df Residuals:                     864                                         
Df Model:                          11                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -2.7508      0.089    -30.760      0.000